<a href="https://colab.research.google.com/github/kyouyap/readability/blob/master/0603_RoBERTa_Torch%7CFIT_CV_0_4759.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
    train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
    test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
    print("train shape",train.shape)
# colaboratory環境ならTrue
elif 'COLAB_GPU' in set(os.environ.keys()):
    train = pd.read_csv("/content/drive/MyDrive/kaggle/readability/train.csv")
    test = pd.read_csv("/content/drive/MyDrive/kaggle/readability/test.csv")
    print("train shape",train.shape)
df=pd.concat([train,test])

train shape (2834, 6)


In [2]:
from sklearn import model_selection
def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.KFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for f, (t_, v_) in enumerate(kf.split(X=data)):
        data.loc[v_, 'kfold'] = f
    return data
train = create_folds(train, num_splits=5)


In [3]:

%matplotlib inline
from glob import glob
import os
import matplotlib.pyplot as plt
import json
from collections import defaultdict
import gc
gc.enable()

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.optimizer import Optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import (
    Dataset, DataLoader, 
    SequentialSampler, RandomSampler
)
!pip install -q transformers
from transformers import AutoConfig
from transformers import (
    get_cosine_schedule_with_warmup, 
    get_cosine_with_hard_restarts_schedule_with_warmup
)
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import MODEL_FOR_TOKEN_CLASSIFICATION_MAPPING
from IPython.display import clear_output
from tqdm import tqdm, trange


     |████████████████████████████████| 2.3MB 7.8MB/s 
     |████████████████████████████████| 3.3MB 50.6MB/s 
     |████████████████████████████████| 901kB 53.9MB/s 


In [5]:
!pip install lda
import lda
from sklearn.feature_extraction.text import CountVectorizer
import joblib
import time

     |████████████████████████████████| 358kB 9.2MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 


In [6]:
def convert_examples_to_features(data, tokenizer, max_len, is_test=False):
    data = data.replace('\n', '')
    tok = tokenizer.encode_plus(
        data, 
        max_length=max_len, 
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    curr_sent = {}
    padding_length = max_len - len(tok['input_ids'])
    curr_sent['input_ids'] = tok['input_ids'] + ([1] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent

In [7]:
class DatasetRetriever(Dataset):
    def __init__(self, data, tokenizer, max_len, is_test=False):
        self.data = data
        if 'excerpt' in self.data.columns:
            self.excerpts = self.data.excerpt.values.tolist()
        else:
            self.excerpts = self.data.text.values.tolist()
        self.targets = self.data.target.values.tolist()
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        excerpt, label = self.excerpts[item], self.targets[item]
        features = convert_examples_to_features(
            excerpt, self.tokenizer, 
            self.max_len, self.is_test
        )
        return {
            'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
            'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
            'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
            'label':torch.tensor(label, dtype=torch.double),
        }

In [8]:
class CommonLitModel(nn.Module):
    def __init__(
        self, 
        model_name, 
        config,  
        multisample_dropout=False,
        output_hidden_states=False
    ):
        super(CommonLitModel, self).__init__()
        self.config = config
        self.roberta = AutoModel.from_pretrained(
            model_name, 
            output_hidden_states=output_hidden_states
        )
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        if multisample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(0.5) for _ in range(5)
            ])
        else:
            self.dropouts = nn.ModuleList([nn.Dropout(0.1)])
        #self.regressor = nn.Linear(config.hidden_size*2, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
        self._init_weights(self.layer_norm)
        self._init_weights(self.regressor)
 
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
 
    def forward(
        self, 
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[1]
        sequence_output = self.layer_norm(sequence_output)
 
        # max-avg head
        # average_pool = torch.mean(sequence_output, 1)
        # max_pool, _ = torch.max(sequence_output, 1)
        # concat_sequence_output = torch.cat((average_pool, max_pool), 1)
 
        # multi-sample dropout
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logits = self.regressor(dropout(sequence_output))
            else:
                logits += self.regressor(dropout(sequence_output))
        
        logits /= len(self.dropouts)
 
        # calculate loss
        loss = None
        if labels is not None:
            # regression task
            loss_fn = torch.nn.MSELoss()
            logits = logits.view(-1).to(labels.dtype)
            loss = torch.sqrt(loss_fn(logits, labels.view(-1)))
        
        output = (logits,) + outputs[1:]
        return ((loss,) + output) if loss is not None else output

In [9]:

class Lamb(Optimizer):
    # Reference code: https://github.com/cybertronai/pytorch-lamb

    def __init__(
        self,
        params,
        lr: float = 1e-3,
        betas = (0.9, 0.999),
        eps: float = 1e-6,
        weight_decay: float = 0,
        clamp_value: float = 10,
        adam: bool = False,
        debias: bool = False,
    ):
        if lr <= 0.0:
            raise ValueError('Invalid learning rate: {}'.format(lr))
        if eps < 0.0:
            raise ValueError('Invalid epsilon value: {}'.format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(
                'Invalid beta parameter at index 0: {}'.format(betas[0])
            )
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(
                'Invalid beta parameter at index 1: {}'.format(betas[1])
            )
        if weight_decay < 0:
            raise ValueError(
                'Invalid weight_decay value: {}'.format(weight_decay)
            )
        if clamp_value < 0.0:
            raise ValueError('Invalid clamp value: {}'.format(clamp_value))

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.clamp_value = clamp_value
        self.adam = adam
        self.debias = debias

        super(Lamb, self).__init__(params, defaults)

    def step(self, closure = None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    msg = (
                        'Lamb does not support sparse gradients, '
                        'please consider SparseAdam instead'
                    )
                    raise RuntimeError(msg)

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(
                        p, memory_format=torch.preserve_format
                    )
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(
                        p, memory_format=torch.preserve_format
                    )

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # Paper v3 does not use debiasing.
                if self.debias:
                    bias_correction = math.sqrt(1 - beta2 ** state['step'])
                    bias_correction /= 1 - beta1 ** state['step']
                else:
                    bias_correction = 1

                # Apply bias to lr to avoid broadcast.
                step_size = group['lr'] * bias_correction

                weight_norm = torch.norm(p.data).clamp(0, self.clamp_value)

                adam_step = exp_avg / exp_avg_sq.sqrt().add(group['eps'])
                if group['weight_decay'] != 0:
                    adam_step.add_(p.data, alpha=group['weight_decay'])

                adam_norm = torch.norm(adam_step)
                if weight_norm == 0 or adam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / adam_norm
                state['weight_norm'] = weight_norm
                state['adam_norm'] = adam_norm
                state['trust_ratio'] = trust_ratio
                if self.adam:
                    trust_ratio = 1

                p.data.add_(adam_step, alpha=-step_size * trust_ratio)

        return loss

In [10]:
def get_optimizer_params(model):
    # 微分学習率とウェイトディケイ
    param_optimizer = list(model.named_parameters())
    learning_rate = 5e-5
    no_decay = ['bias', 'gamma', 'beta']
    group1=['layer.0.','layer.1.','layer.2.','layer.3.']
    group2=['layer.4.','layer.5.','layer.6.','layer.7.']    
    group3=['layer.8.','layer.9.','layer.10.','layer.11.']
    group_all=['layer.0.','layer.1.','layer.2.','layer.3.','layer.4.','layer.5.','layer.6.','layer.7.','layer.8.','layer.9.','layer.10.','layer.11.']
    optimizer_parameters = [
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay_rate': 0.01},
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay_rate': 0.01, 'lr': learning_rate/2.6},
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay_rate': 0.01, 'lr': learning_rate},
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay_rate': 0.01, 'lr': learning_rate*2.6},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay_rate': 0.0},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay_rate': 0.0, 'lr': learning_rate/2.6},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay_rate': 0.0, 'lr': learning_rate},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay_rate': 0.0, 'lr': learning_rate*2.6},
        {'params': [p for n, p in model.named_parameters() if "roberta" not in n], 'lr':1e-3, "momentum" : 0.99},
    ]
    return optimizer_parameters

In [11]:

def make_model(model_name='../input/robertaitpt/', num_labels=1):
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    config = AutoConfig.from_pretrained(model_name)
    config.update({'num_labels':num_labels})
    model = CommonLitModel(model_name, config=config)
    return model, tokenizer

def make_optimizer(model, optimizer_name="AdamW"):
    optimizer_grouped_parameters = get_optimizer_params(model)
    kwargs = {
            'lr':3e-5,
            'weight_decay':0
    }
    if optimizer_name == "LAMB":
        optimizer = Lamb(optimizer_grouped_parameters, **kwargs)
        return optimizer
    elif optimizer_name == "Adam":
        from torch.optim import Adam
        optimizer = Adam(optimizer_grouped_parameters, **kwargs)
        return optimizer
    elif optimizer_name == "AdamW":
        from torch.optim import AdamW
        optimizer = AdamW(optimizer_grouped_parameters, **kwargs)
        return optimizer
    else:
        raise Exception('Unknown optimizer: {}'.format(optimizer_name))

def make_scheduler(optimizer, decay_name='cosine_warmup', t_max=10):
    if decay_name == 'step':
        scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer,
            milestones=[30, 60, 90],
            gamma=0.1
        )
    elif decay_name == 'cosine':
        scheduler = lrs.CosineAnnealingLR(
            optimizer,
            T_max=t_max
        )
    elif decay_name == "cosine_warmup":
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=t_max
        )
    else:
        raise Exception('Unknown lr scheduler: {}'.format(decay_type))    
    return scheduler    

def make_loader(
    data, 
    tokenizer, 
    max_len,
    batch_size,
    fold=0
):
    train_set, valid_set = data[data['kfold']!=fold], data[data['kfold']==fold]
    train_dataset = DatasetRetriever(train_set, tokenizer, max_len)
    valid_dataset = DatasetRetriever(valid_set, tokenizer, max_len)

    train_sampler = RandomSampler(train_dataset)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        sampler=train_sampler, 
        pin_memory=True, 
        drop_last=False, 
        num_workers=4
    )

    valid_sampler = SequentialSampler(valid_dataset)
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size=batch_size // 2, 
        sampler=valid_sampler, 
        pin_memory=True, 
        drop_last=False, 
        num_workers=4
    )

    return train_loader, valid_loader

In [12]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.max = 0
        self.min = 1e5

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        if val > self.max:
            self.max = val
        if val < self.min:
            self.min = val

In [13]:
class Trainer:
    def __init__(self, model, optimizer, scheduler, scalar=None, log_interval=1, evaluate_interval=1):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.scalar = scalar
        self.log_interval = log_interval
        self.evaluate_interval = evaluate_interval
        self.evaluator = Evaluator(self.model, self.scalar)

    def train(self, train_loader, valid_loader, epoch, 
              result_dict, tokenizer, fold):
        total_loss = 0
        count = 0

        losses = AverageMeter()

        self.model.train()
        
        for batch_idx, batch_data in enumerate(train_loader):
            input_ids, attention_mask, token_type_ids, labels = batch_data['input_ids'], \
                batch_data['attention_mask'], batch_data['token_type_ids'], batch_data['label']
            input_ids, attention_mask, token_type_ids, labels = \
                input_ids.cuda(), attention_mask.cuda(), token_type_ids.cuda(), labels.cuda()

            
            if self.scalar is not None:
                with torch.cuda.amp.autocast():
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels
                    )
            else:
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=labels
                )
            
#             torch.nn.utils.clip_grad_norm_(
#                 parameters=self.model.parameters(), 
#                 max_norm=1.0
#             )

            loss, logits = outputs[:2]
            total_loss += loss.tolist()
            count += labels.size(0)

            logits = logits.cpu().detach().numpy()
            losses.update(loss.item(), input_ids.size(0))
            

            self.optimizer.zero_grad()
            if self.scalar is not None:
                self.scalar.scale(loss).backward()
                self.scalar.step(self.optimizer)
                self.scalar.update()
            else:
                loss.backward()
                self.optimizer.step()

            if batch_idx % self.log_interval == 0:
                _s = str(len(str(len(train_loader.sampler))))
                ret = [
                    ('epoch: {:0>3} [{: >' + _s + '}/{} ({: >3.0f}%)]').format(epoch, count, len(train_loader.sampler), 100 * count / len(train_loader.sampler)),
                    'train_loss: {: >4.5f}'.format(total_loss / count),
                ]
                print(', '.join(ret))
            
            if batch_idx % self.evaluate_interval == 0:
                result_dict = self.evaluator.evaluate(
                    valid_loader, 
                    epoch, 
                    result_dict, 
                    tokenizer
                )
                if result_dict['val_loss'][-1] < result_dict['best_val_loss']:
                    print("{} epoch, best epoch was updated! valid_loss: {: >4.5f}".format(epoch, result_dict['val_loss'][-1]))
                    result_dict["best_val_loss"] = result_dict['val_loss'][-1]
                    torch.save(self.model.state_dict(), f"model{fold}.bin")

        self.scheduler.step()
        result_dict['train_loss'].append(losses.avg)
        return result_dict

In [14]:
class Evaluator:
    def __init__(self, model, scalar=None):
        self.model = model
        self.scalar = scalar
    
    def worst_result(self):
        ret = {
            'loss':float('inf'),
            'accuracy':0.0
        }
        return ret

    def result_to_str(self, result):
        ret = [
            'epoch: {epoch:0>3}',
            'loss: {loss: >4.2e}'
        ]
        for metric in self.evaluation_metrics:
            ret.append('{}: {}'.format(metric.name, metric.fmtstr))
        return ', '.join(ret).format(**result)

    def save(self, result):
        with open('result_dict.json', 'w') as f:
            f.write(json.dumps(result, sort_keys=True, indent=4, ensure_ascii=False))
    
    def load(self):
        result = self.worst_result
        if os.path.exists('result_dict.json'):
            with open('result_dict.json', 'r') as f:
                try:
                    result = json.loads(f.read())
                except:
                    pass
        return result

    def evaluate(self, data_loader, epoch, result_dict, tokenizer):
        losses = AverageMeter()

        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch_idx, batch_data in enumerate(data_loader):
                input_ids, attention_mask, token_type_ids, labels = batch_data['input_ids'], \
                    batch_data['attention_mask'], batch_data['token_type_ids'], batch_data['label']
                input_ids, attention_mask, token_type_ids, labels = input_ids.cuda(), \
                    attention_mask.cuda(), token_type_ids.cuda(), labels.cuda()
                
                if self.scalar is not None:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            labels=labels
                        )
                else:
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels
                    )
                
                loss, logits = outputs[:2]
                losses.update(loss.item(), input_ids.size(0))

        print('----Validation Results Summary----')
        print('Epoch: [{}] train_loss: {: >4.5f}'.format(epoch, losses.avg))

        result_dict['val_loss'].append(losses.avg)        
        return result_dict

In [15]:
def config(fold=0):
    torch.manual_seed(2021)
    torch.cuda.manual_seed(2021)
    torch.cuda.manual_seed_all(2021)
    epochs = 8
    max_len = 250
    batch_size = 24

    model, tokenizer = make_model(model_name='/content/drive/MyDrive/kaggle/readability/roberta-e1', num_labels=1)
    optimizer = make_optimizer(model, "AdamW")
    scheduler = make_scheduler(optimizer, t_max=epochs)
    train_loader, valid_loader = make_loader(
        train, tokenizer, max_len=max_len,
        batch_size=batch_size, fold=fold
    )

    if torch.cuda.device_count() >= 1:
        print('Model pushed to {} GPU(s), type {}.'.format(
            torch.cuda.device_count(), 
            torch.cuda.get_device_name(0))
        )
        model = model.cuda() 
    else:
        raise ValueError('CPU training is not supported')

    # scaler = torch.cuda.amp.GradScaler()
    scaler = None

    result_dict = {
        'epoch':[], 
        'train_loss': [], 
        'val_loss' : [], 
        'test_loss': [],
        'best_val_loss': np.inf
    }
    return (
        model, tokenizer, 
        optimizer, scheduler, 
        scaler, train_loader, 
        valid_loader, result_dict, 
        epochs
    )

In [16]:
def run(fold=0):
    model, tokenizer, optimizer, scheduler, scaler, \
        train_loader, valid_loader, result_dict, epochs = config(fold)
    import time
    trainer = Trainer(model, optimizer, scheduler, scaler)
    train_time_list = []

    for epoch in range(epochs):
        result_dict['epoch'] = epoch

        torch.cuda.synchronize()
        tic1 = time.time()

        result_dict = trainer.train(train_loader, valid_loader, epoch, 
                                    result_dict, tokenizer, fold)

        torch.cuda.synchronize()
        tic2 = time.time() 
        train_time_list.append(tic2 - tic1)

    torch.cuda.empty_cache()
    del model, tokenizer, optimizer, scheduler, \
        scaler, train_loader, valid_loader,
    gc.collect()
    return result_dict

In [17]:
best_scores=[]
for fold in range(5):
    print('----')
    print(f'FOLD: {fold}')
    result=run(fold)
    print('----')
    print()
    best_scores.append(result["best_val_loss"])
print(f"CV:{np.mean(best_scores)}")

----
FOLD: 0


Some weights of the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should pr

Model pushed to 1 GPU(s), type Tesla P100-PCIE-16GB.
epoch: 000 [  24/2267 (  1%)], train_loss: 0.05160
----Validation Results Summary----
Epoch: [0] train_loss: 1.49696
0 epoch, best epoch was updated! valid_loss: 1.49696
epoch: 000 [  48/2267 (  2%)], train_loss: 0.05880
----Validation Results Summary----
Epoch: [0] train_loss: 1.17879
0 epoch, best epoch was updated! valid_loss: 1.17879
epoch: 000 [  72/2267 (  3%)], train_loss: 0.05544
----Validation Results Summary----
Epoch: [0] train_loss: 1.01946
0 epoch, best epoch was updated! valid_loss: 1.01946
epoch: 000 [  96/2267 (  4%)], train_loss: 0.04957
----Validation Results Summary----
Epoch: [0] train_loss: 1.05827
epoch: 000 [ 120/2267 (  5%)], train_loss: 0.04670
----Validation Results Summary----
Epoch: [0] train_loss: 1.04446
epoch: 000 [ 144/2267 (  6%)], train_loss: 0.04368
----Validation Results Summary----
Epoch: [0] train_loss: 1.03347
epoch: 000 [ 168/2267 (  7%)], train_loss: 0.04388
----Validation Results Summary----


Some weights of the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should pr

Model pushed to 1 GPU(s), type Tesla P100-PCIE-16GB.
epoch: 000 [  24/2267 (  1%)], train_loss: 0.04927
----Validation Results Summary----
Epoch: [0] train_loss: 1.46897
0 epoch, best epoch was updated! valid_loss: 1.46897
epoch: 000 [  48/2267 (  2%)], train_loss: 0.05378
----Validation Results Summary----
Epoch: [0] train_loss: 1.20427
0 epoch, best epoch was updated! valid_loss: 1.20427
epoch: 000 [  72/2267 (  3%)], train_loss: 0.05680
----Validation Results Summary----
Epoch: [0] train_loss: 0.96826
0 epoch, best epoch was updated! valid_loss: 0.96826
epoch: 000 [  96/2267 (  4%)], train_loss: 0.05353
----Validation Results Summary----
Epoch: [0] train_loss: 1.11620
epoch: 000 [ 120/2267 (  5%)], train_loss: 0.05326
----Validation Results Summary----
Epoch: [0] train_loss: 1.10734
epoch: 000 [ 144/2267 (  6%)], train_loss: 0.05130
----Validation Results Summary----
Epoch: [0] train_loss: 1.03940
epoch: 000 [ 168/2267 (  7%)], train_loss: 0.05037
----Validation Results Summary----


...

Model pushed to 1 GPU(s), type Tesla P100-PCIE-16GB.
epoch: 000 [  24/2267 (  1%)], train_loss: 0.05974
----Validation Results Summary----
Epoch: [0] train_loss: 1.48241
0 epoch, best epoch was updated! valid_loss: 1.48241
epoch: 000 [  48/2267 (  2%)], train_loss: 0.05926
----Validation Results Summary----
Epoch: [0] train_loss: 1.23934
0 epoch, best epoch was updated! valid_loss: 1.23934
epoch: 000 [  72/2267 (  3%)], train_loss: 0.06064
----Validation Results Summary----
Epoch: [0] train_loss: 0.97669
0 epoch, best epoch was updated! valid_loss: 0.97669
epoch: 000 [  96/2267 (  4%)], train_loss: 0.05320
----Validation Results Summary----
Epoch: [0] train_loss: 1.08749
epoch: 000 [ 120/2267 (  5%)], train_loss: 0.05198
----Validation Results Summary----
Epoch: [0] train_loss: 1.13221
epoch: 000 [ 144/2267 (  6%)], train_loss: 0.05178
----Validation Results Summary----
Epoch: [0] train_loss: 1.07035
epoch: 000 [ 168/2267 (  7%)], train_loss: 0.05179
----Validation Results Summary----


Some weights of the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should pr

Model pushed to 1 GPU(s), type Tesla P100-PCIE-16GB.
epoch: 000 [  24/2267 (  1%)], train_loss: 0.04603
----Validation Results Summary----
Epoch: [0] train_loss: 1.43316
0 epoch, best epoch was updated! valid_loss: 1.43316
epoch: 000 [  48/2267 (  2%)], train_loss: 0.05285
----Validation Results Summary----
Epoch: [0] train_loss: 1.08554
0 epoch, best epoch was updated! valid_loss: 1.08554
epoch: 000 [  72/2267 (  3%)], train_loss: 0.04847
----Validation Results Summary----
Epoch: [0] train_loss: 0.99668
0 epoch, best epoch was updated! valid_loss: 0.99668
epoch: 000 [  96/2267 (  4%)], train_loss: 0.04662
----Validation Results Summary----
Epoch: [0] train_loss: 1.02149
epoch: 000 [ 120/2267 (  5%)], train_loss: 0.04586
----Validation Results Summary----
Epoch: [0] train_loss: 0.99742
epoch: 000 [ 144/2267 (  6%)], train_loss: 0.04436
----Validation Results Summary----
Epoch: [0] train_loss: 1.00208
epoch: 000 [ 168/2267 (  7%)], train_loss: 0.04289
----Validation Results Summary----


Some weights of the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/kaggle/readability/roberta-e1 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should pr

Model pushed to 1 GPU(s), type Tesla P100-PCIE-16GB.
epoch: 000 [  24/2268 (  1%)], train_loss: 0.05549
----Validation Results Summary----
Epoch: [0] train_loss: 1.46877
0 epoch, best epoch was updated! valid_loss: 1.46877
epoch: 000 [  48/2268 (  2%)], train_loss: 0.06291
----Validation Results Summary----
Epoch: [0] train_loss: 1.22587
0 epoch, best epoch was updated! valid_loss: 1.22587
epoch: 000 [  72/2268 (  3%)], train_loss: 0.05777
----Validation Results Summary----
Epoch: [0] train_loss: 0.98442
0 epoch, best epoch was updated! valid_loss: 0.98442
epoch: 000 [  96/2268 (  4%)], train_loss: 0.05418
----Validation Results Summary----
Epoch: [0] train_loss: 1.05069
epoch: 000 [ 120/2268 (  5%)], train_loss: 0.05252
----Validation Results Summary----
Epoch: [0] train_loss: 1.09725
epoch: 000 [ 144/2268 (  6%)], train_loss: 0.04944
----Validation Results Summary----
Epoch: [0] train_loss: 1.06068
epoch: 000 [ 168/2268 (  7%)], train_loss: 0.04895
----Validation Results Summary----


In [18]:
!mkdir /content/drive/MyDrive/kaggle/readability/roberta
!cp /content/model0.bin /content/drive/MyDrive/kaggle/readability/roberta
!cp /content/model1.bin /content/drive/MyDrive/kaggle/readability/roberta
!cp /content/model2.bin /content/drive/MyDrive/kaggle/readability/roberta
!cp /content/model2.bin /content/drive/MyDrive/kaggle/readability/roberta
!cp /content/model3.bin /content/drive/MyDrive/kaggle/readability/roberta
!cp /content/model4.bin /content/drive/MyDrive/kaggle/readability/roberta

mkdir: cannot create directory ‘/content/drive/MyDrive/kaggle/readability/robertalda’: File exists
